# CytoSoftware-Red Particle Quantification (RPQ)

This notebook detects **red fluorescent particles** in MP4 videos **without cell segmentation**, and plots counts vs time.

Enhancements added:
- Optional rolling‑ball background subtraction (`BACKGROUND_METHOD`)
- Per‑particle feature export (`red_blob_features.csv`)
- Optional local SNR filter (`SNR_MIN`)
- 95% CI uncertainty band on time‑series plot
- Saved run configuration (`run_config.json`) for reproducibility


In [1]:
# Cell 1: Install dependencies (run once per session)
# In Colab, run this cell then restart runtime if asked.


!pip -q install opencv-python-headless scikit-image pandas tqdm ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00


In [2]:
# Cell 2: Imports
import os, math, warnings, json
import numpy as np
import pandas as pd
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import seaborn as sns
from skimage import util
from skimage.feature import blob_log
from skimage.measure import regionprops, label
from skimage.morphology import disk, opening

import ipywidgets as widgets
from IPython.display import display
from google.colab import files

# Optional for confidence intervals
try:
    from scipy.stats import poisson
except Exception:  # pragma: no cover
    poisson = None

warnings.filterwarnings('ignore')


# **UPLOAD MP4**

In [3]:
# Cell 3: Upload MP4 (Colab or local Jupyter)

video_path = None

def _is_colab():
    try:
        import google.colab  # noqa
        return True
    except Exception:
        return False

from google.colab import files

download_msg = widgets.HTML(
    "<b>Analysis completed.</b><br>"
    "You can download the result CSV below:<br>"
    "<span style='color:gray; font-size:90%'>"
    "CSV file contains the number of red particles detected per frame over time."
    "</span>"
)

download_ui_out = widgets.Output()

btn_download = widgets.Button(
    description="Download CSV",
    button_style="primary",
    icon="download",
    disabled=True
)

def _on_download_clicked(b):
    if "csv_path" in globals() and os.path.exists(csv_path):
        from google.colab import files
        files.download(csv_path)

btn_download.on_click(_on_download_clicked)

def run_after_upload(video_path):
    import cv2, os, warnings
    global VIDEO_FRAME_COUNT, VIDEO_FPS
    global PROCESS_EVERY_N_FRAMES, MAX_FRAMES_TO_ANALYZE
    global BG_BLUR_SIGMA, BACKGROUND_METHOD, ROLLING_BALL_RADIUS
    global SNR_MIN, RED_DOM_RATIO, RED_MIN_ABS
    global LOG_SIGMA_MIN, LOG_SIGMA_MAX, LOG_NUM_SIGMAS, LOG_THRESH, ROBUST_K
    global MIN_PARTICLE_AREA, MAX_PARTICLE_AREA
    global DEFAULT_TOTAL_OBS_MIN, DEFAULT_INTERVAL_MIN
    global out_dir, csv_path, annotated_mp4_path

    print(" ")
    print("_______________________________")
    print("Setting")

    # ------------------------------
    # Settings (tune if needed)
    # ------------------------------

    if video_path is None:
        raise ValueError('video_path is None.')

    if not os.path.exists(video_path):
        raise FileNotFoundError(f'Video file not found: {video_path}')

    cap0 = cv2.VideoCapture(video_path)
    if not cap0.isOpened():
        raise IOError(f'Cannot open video: {video_path}')

    VIDEO_FRAME_COUNT = int(cap0.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    VIDEO_FPS = float(cap0.get(cv2.CAP_PROP_FPS) or 0)
    if (not VIDEO_FPS) or VIDEO_FPS < 1:
        warnings.warn('FPS invalid, defaulting to 25.')
        VIDEO_FPS = 25.0
    cap0.release()

    print(f'Video OK: {VIDEO_FRAME_COUNT} frames, {VIDEO_FPS:.3f} fps')

    # Performance
    PROCESS_EVERY_N_FRAMES = 1
    MAX_FRAMES_TO_ANALYZE = None

    # Red enhancement / background
    BG_BLUR_SIGMA = 8
    BACKGROUND_METHOD = "gaussian"
    ROLLING_BALL_RADIUS = 25
    SNR_MIN = 0.0
    RED_DOM_RATIO = 1.6
    RED_MIN_ABS = 0.08

    # Blob detection
    LOG_SIGMA_MIN = 1.0
    LOG_SIGMA_MAX = 4.5
    LOG_NUM_SIGMAS = 8
    LOG_THRESH = 0.03
    ROBUST_K = 3.0

    MIN_PARTICLE_AREA = 6
    MAX_PARTICLE_AREA = 260

    # Time defaults
    DEFAULT_TOTAL_OBS_MIN = 240.0
    DEFAULT_INTERVAL_MIN = 30.0

    # Output
    out_dir = "/mnt/data/red_particles_no_seg_out"
    os.makedirs(out_dir, exist_ok=True)
    csv_path = os.path.join(out_dir, "red_particles_by_time.csv")
    annotated_mp4_path = os.path.join(out_dir, "annotated_red_particles.mp4")

    # print("Setting completed successfully.")

def iter_video_frames(path, every_n=1, max_frames=None):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    if not fps or fps < 1:
        print("Warning: Could not determine FPS, using 25.0 as default")
        fps = 25.0
    idx = 0
    kept = 0
    while True:
        ok, frame_bgr = cap.read()
        if not ok:
            break
        if idx % every_n == 0:
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            yield kept, idx, frame_rgb, fps
            kept += 1
            if max_frames is not None and kept >= max_frames:
                break
        idx += 1
    cap.release()

def _estimate_background(enh):
    """Estimate smooth background in the red-enhanced image."""
    method = str(BACKGROUND_METHOD).lower()
    if method == "rolling_ball":
        rad = max(1, int(round(ROLLING_BALL_RADIUS)))
        try:
            bg = opening(enh, disk(rad))
        except Exception:
            # Fallback to gaussian if morphology not available
            bg = cv2.GaussianBlur(enh, (0,0), BG_BLUR_SIGMA)
    else:
        bg = cv2.GaussianBlur(enh, (0,0), BG_BLUR_SIGMA)
    return bg

def red_enhance(frame_rgb):
    # normalize to 0-1 float
    f = util.img_as_float32(frame_rgb)
    r, g, b = f[...,0], f[...,1], f[...,2]
    # red dominance image
    enh = r - 0.5*(g + b)
    enh = np.clip(enh, 0, 1)
    # subtract smooth background
    bg = _estimate_background(enh)
    enh2 = np.clip(enh - bg, 0, 1)
    return enh2, r, g, b, bg

def robust_threshold(img, base=0.03, k=3.0):
    med = np.median(img)
    mad = np.median(np.abs(img - med)) + 1e-6
    thr = max(base, med + k*mad)
    return thr

def detect_red_blobs(frame_rgb):
    enh, r, g, b, bg_img = red_enhance(frame_rgb)
    thr = robust_threshold(enh, base=LOG_THRESH, k=ROBUST_K)

    # Defensive parameter validation
    sigma_min = max(0.5, float(LOG_SIGMA_MIN))
    sigma_max = float(LOG_SIGMA_MAX)
    if sigma_max < sigma_min:
        sigma_max = sigma_min
    num_sigma = int(LOG_NUM_SIGMAS) if int(LOG_NUM_SIGMAS) >= 1 else 1

    blobs = blob_log(enh, min_sigma=sigma_min, max_sigma=sigma_max,
                     num_sigma=num_sigma, threshold=thr)
    # blobs: (y, x, sigma)
    accepted = []
    for (y, x, s) in blobs:
        y, x = int(round(y)), int(round(x))
        rad = max(2, int(round(math.sqrt(2)*s)))
        y0, y1 = max(0, y-rad), min(enh.shape[0], y+rad+1)
        x0, x1 = max(0, x-rad), min(enh.shape[1], x+rad+1)

        patch_enh = enh[y0:y1, x0:x1]
        patch_r = r[y0:y1, x0:x1]
        patch_g = g[y0:y1, x0:x1]
        patch_b = b[y0:y1, x0:x1]

        # reject if enhancement too weak
        if float(patch_enh.max()) < RED_MIN_ABS:
            continue

        mean_r = float(patch_r.mean())
        mean_g = float(patch_g.mean()) + 1e-6
        mean_b = float(patch_b.mean()) + 1e-6
        dom_rg = mean_r/mean_g
        dom_rb = mean_r/mean_b
        if not (dom_rg >= RED_DOM_RATIO and dom_rb >= RED_DOM_RATIO):
            continue

        # circular mask in patch for size + local stats
        yy, xx = np.ogrid[y0:y1, x0:x1]
        mask = (yy-y)**2 + (xx-x)**2 <= rad**2
        area = int(mask.sum())
        if area < MIN_PARTICLE_AREA or area > MAX_PARTICLE_AREA:
            continue

        core_vals = patch_enh[mask]
        bg_vals = patch_enh[~mask]
        if bg_vals.size < 5:
            bg_vals = patch_enh.ravel()
        bg_mean = float(bg_vals.mean())
        bg_std = float(bg_vals.std()) + 1e-6
        peak = float(core_vals.max()) if core_vals.size else float(patch_enh.max())
        snr = float((peak - bg_mean) / bg_std)
        if SNR_MIN and snr < SNR_MIN:
            continue

        accepted.append({
            "y": y, "x": x, "sigma": float(s), "radius": int(rad),
            "area": int(area),
            "mean_r": float(mean_r), "mean_g": float(mean_g-1e-6), "mean_b": float(mean_b-1e-6),
            "dom_rg": float(dom_rg), "dom_rb": float(dom_rb),
            "peak_enh": float(peak),
            "bg_mean_enh": float(bg_mean), "bg_std_enh": float(bg_std),
            "snr": float(snr)
        })
    return accepted

def overlay_blobs(frame_rgb, blobs):
    out = frame_rgb.copy()
    for blob in blobs:
        y = blob["y"]; x = blob["x"]; rad = blob["radius"]
        cv2.circle(out, (int(x), int(y)), int(rad), (255,0,0), 2)
    return out

_globals_defaults = {
    "_fixed_preview_frame": None,
    "_fixed_preview_orig_i": None,
    "_fixed_preview_kept_i": None,
    "preview_frame": None,
    "DEFAULT_TOTAL_OBS_MIN": globals().get("DEFAULT_TOTAL_OBS_MIN", None),
    "DEFAULT_INTERVAL_MIN": globals().get("DEFAULT_INTERVAL_MIN", None),
}

for _k, _v in _globals_defaults.items():
    if _k not in globals():
        globals()[_k] = _v

# optional: clean up helper name
del _globals_defaults

def run_analysis():
    print(" ")
    print("_______________________________")
    print("Running Analysis ...")

    import ipywidgets as widgets
    from IPython.display import display, clear_output
    global VIDEO_FRAME_COUNT, VIDEO_FPS
    global PROCESS_EVERY_N_FRAMES, MAX_FRAMES_TO_ANALYZE
    global BG_BLUR_SIGMA, BACKGROUND_METHOD, ROLLING_BALL_RADIUS
    global SNR_MIN, RED_DOM_RATIO, RED_MIN_ABS
    global LOG_SIGMA_MIN, LOG_SIGMA_MAX, LOG_NUM_SIGMAS, LOG_THRESH, ROBUST_K
    global MIN_PARTICLE_AREA, MAX_PARTICLE_AREA
    global DEFAULT_TOTAL_OBS_MIN, DEFAULT_INTERVAL_MIN
    global out_dir, csv_path, annotated_mp4_path

    def _read_last_frame(path):
        cap = cv2.VideoCapture(path)
        n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if n <= 0:
            ok, fr = cap.read()
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, n - 1)
            ok, fr = cap.read()
        cap.release()
        if not ok:
            raise IOError(f"Cannot read last frame from {path}")
        return cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)

    preview_frame = _read_last_frame(video_path)

    # Lock the preview frame once the max-red frame is found.
    # Parameter changes will update detections on this same frame, not re-select a new frame.
    _fixed_preview_frame = None
    _fixed_preview_orig_i = None
    _fixed_preview_kept_i = None


    # Sliders
    w_bg_sigma = widgets.FloatSlider(value=BG_BLUR_SIGMA, min=1, max=30, step=0.5, description="BG sigma")
    w_red_dom  = widgets.FloatSlider(value=RED_DOM_RATIO, min=1.0, max=3.0, step=0.05, description="Red dom")
    w_red_abs  = widgets.FloatSlider(value=RED_MIN_ABS, min=0.00, max=0.40, step=0.01, description="Red min abs")

    w_sig_min  = widgets.FloatSlider(value=LOG_SIGMA_MIN, min=0.5, max=8, step=0.5, description="σ min")
    w_sig_max  = widgets.FloatSlider(value=LOG_SIGMA_MAX, min=2, max=20, step=0.5, description="σ max")
    # keep sigma_max >= sigma_min
    if LOG_SIGMA_MAX < LOG_SIGMA_MIN:
        LOG_SIGMA_MAX = LOG_SIGMA_MIN + 0.5
        w_sig_max.value = LOG_SIGMA_MAX
    w_sig_num  = widgets.IntSlider(value=LOG_NUM_SIGMAS, min=3, max=20, step=1, description="#σ")

    w_log_thr  = widgets.FloatSlider(value=LOG_THRESH, min=0.001, max=0.20, step=0.002, description="LoG thr")
    w_rob_k    = widgets.FloatSlider(value=ROBUST_K, min=0.5, max=6.0, step=0.1, description="Robust k")

    w_area_min = widgets.IntSlider(value=MIN_PARTICLE_AREA, min=1, max=200, step=1, description="Area min")
    w_area_max = widgets.IntSlider(value=MAX_PARTICLE_AREA, min=5, max=3000, step=5, description="Area max")

    btn_preview = widgets.Button(description="Apply & Preview", button_style="success", icon="check")
    btn_run = widgets.Button(description="Run analysis", button_style="success", icon="play", disabled=True, tooltip="Run full analysis with current parameters")

    _preview_ready = {"ok": False}


    out = widgets.Output()

    def _mark_dirty(change=None):
        # Any parameter change invalidates the previous preview and any prior results
        _preview_ready["ok"] = False
        btn_run.disabled = True
        btn_download.disabled = True
        with download_ui_out:
            download_ui_out.clear_output(wait=True)

    for _w in [w_bg_sigma, w_red_dom, w_red_abs, w_sig_min, w_sig_max, w_sig_num, w_log_thr, w_rob_k, w_area_min, w_area_max]:
        _w.observe(_mark_dirty, names="value")


    def _apply_globals():
        global BG_BLUR_SIGMA, RED_DOM_RATIO, RED_MIN_ABS
        global LOG_SIGMA_MIN, LOG_SIGMA_MAX, LOG_NUM_SIGMAS, LOG_THRESH, ROBUST_K
        global MIN_PARTICLE_AREA, MAX_PARTICLE_AREA

        BG_BLUR_SIGMA   = float(w_bg_sigma.value)
        RED_DOM_RATIO   = float(w_red_dom.value)
        RED_MIN_ABS     = float(w_red_abs.value)

        LOG_SIGMA_MIN   = float(w_sig_min.value)
        LOG_SIGMA_MAX   = float(w_sig_max.value)
        # keep sigma_max >= sigma_min
        if LOG_SIGMA_MAX < LOG_SIGMA_MIN:
            LOG_SIGMA_MAX = LOG_SIGMA_MIN + 0.5
            w_sig_max.value = LOG_SIGMA_MAX
        LOG_NUM_SIGMAS  = int(w_sig_num.value)
        LOG_THRESH      = float(w_log_thr.value)
        ROBUST_K        = float(w_rob_k.value)

        MIN_PARTICLE_AREA = int(w_area_min.value)
        MAX_PARTICLE_AREA = int(w_area_max.value)


    def _do_preview(_=None):
        global LOG_SIGMA_MIN, LOG_SIGMA_MAX, preview_frame
        global DEFAULT_TOTAL_OBS_MIN, DEFAULT_INTERVAL_MIN
        global preview_frame_index, df_blobs


        with out:
            # ---- hide download button immediately ----
            btn_download.disabled = True
            with download_ui_out:
                download_ui_out.clear_output(wait=True)

            clear_output(wait=True)
            _apply_globals()

            if video_path is None or not os.path.exists(video_path):
                print("No video loaded for preview.")
                return

            global _fixed_preview_frame, _fixed_preview_orig_i, _fixed_preview_kept_i

            if _fixed_preview_frame is None:
                # Scan a subset of frames and pick the one with the most detected red particles (run once)
                max_preview_red = -1
                best_frame = None
                best_blobs = None
                best_orig_i = None
                best_kept_i = None

                # Keep preview fast: cap the number of scanned frames
                PREVIEW_SCAN_MAX_FRAMES = 200

                for kept_i, orig_i, frame_rgb, fps in iter_video_frames(
                        video_path,
                        every_n=1,  # preview scans consecutive frames regardless of skipping
                        max_frames=PREVIEW_SCAN_MAX_FRAMES):
                    b = detect_red_blobs(frame_rgb)
                    if len(b) > max_preview_red:
                        max_preview_red = len(b)
                        best_frame = frame_rgb.copy()
                        best_orig_i = orig_i
                        best_kept_i = kept_i

                if best_frame is None:
                    print("Preview not available (no frames read).")
                    return

                # Lock this frame for later previews
                _fixed_preview_frame = best_frame
                _fixed_preview_orig_i = best_orig_i
                _fixed_preview_kept_i = best_kept_i
            else:
                best_frame = _fixed_preview_frame
                best_orig_i = _fixed_preview_orig_i
                best_kept_i = _fixed_preview_kept_i

            # Always re-detect on the fixed frame with current parameters
            best_blobs = detect_red_blobs(best_frame)

            if best_frame is None:
                print("Preview not available (no frames read).")
                return

            preview_frame = best_frame
            blobs = best_blobs

            fig = plt.figure(figsize=(7,7))
            plt.imshow(preview_frame)
            ax = plt.gca()
            for blob in blobs:
                y = blob['y']; x = blob['x']; rad = blob['radius']
                rad = int(rad)
                circ = Circle((x, y), rad, color='white', fill=False, linewidth=2)
                ax.add_patch(circ)
            plt.title(f"Preview on max-red frame (orig={best_orig_i}, used={best_kept_i}): n_red = {len(blobs)}")
            plt.axis("off")
            plt.show()
            print(" ")
            print("_______________________________")
            print("Applied parameters:")
            print(f"  BG_BLUR_SIGMA={BG_BLUR_SIGMA}")
            print(f"  RED_DOM_RATIO={RED_DOM_RATIO}, RED_MIN_ABS={RED_MIN_ABS}")
            print(f"  LOG_SIGMA_MIN={LOG_SIGMA_MIN}, LOG_SIGMA_MAX={LOG_SIGMA_MAX}, LOG_NUM_SIGMAS={LOG_NUM_SIGMAS}")
            # keep sigma_max >= sigma_min
            if LOG_SIGMA_MAX < LOG_SIGMA_MIN:
                LOG_SIGMA_MAX = LOG_SIGMA_MIN + 0.5
                w_sig_max.value = LOG_SIGMA_MAX
            print(f"  LOG_THRESH={LOG_THRESH}, ROBUST_K={ROBUST_K}")
            print(f"  MIN_PARTICLE_AREA={MIN_PARTICLE_AREA}, MAX_PARTICLE_AREA={MAX_PARTICLE_AREA}")

            _preview_ready["ok"] = True
            btn_run.disabled = False
            print(" ")
            print("Preview complete. Click 'Run analysis' to process the full video with these parameters.")

        # update state: keep download disabled until analysis is run
        btn_download.disabled = True
        with download_ui_out:
            download_ui_out.clear_output(wait=True)



    def _run_analysis(_=None):
        """Run the full RPQ analysis using the last previewed parameters."""
        with out:
            clear_output(wait=True)

            if not _preview_ready.get("ok", False):
                print("Please click 'Apply & Preview' first (and re-preview after changing parameters).")
                return

            # Sync globals from widgets (safe even if preview already applied them)
            _apply_globals()

            # Disable immediately to enforce Preview -> Run sequence
            btn_run.disabled = True
            _preview_ready["ok"] = False

            # Hide download UI until analysis finishes
            btn_download.disabled = True
            with download_ui_out:
                download_ui_out.clear_output(wait=True)

            print(" ")
            print("_______________________________")
            print("Running Analysis ...")

            state = run_red_particle_analysis()   # returns dict
            display(state["df"].head())

            print(" ")
            print("_______________________________")
            print("Plot + Save annotated video...")
            run_save_and_plot(state)

            # Show download button AFTER analysis finished
            if "csv_path" in globals() and os.path.exists(csv_path):
                btn_download.disabled = False
                with download_ui_out:
                    download_ui_out.clear_output(wait=True)
                    print(" ")
                    print("_______________________________")
                    display(widgets.VBox([download_msg, btn_download]))


    def run_red_particle_analysis():
        """
        Runs analysis (previously Cell 7 analysis). Returns a dict with all outputs
        needed by the plotting/saving step.
        """
        print(" ")
        print("_______________________________")
        print("Red particle analysis...")

        results = []
        blob_rows = []  # per-particle features

        # Video writer variables
        annotated_frames = []
        writer = None
        annotated_video_written = False


        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()

        # Input validation
        if video_path is None:
            raise ValueError("video_path is None. Upload an MP4 or set video_path before running analysis.")
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
        if not video_path.lower().endswith((".mp4", ".MP4", ".avi", ".AVI")):
            print(f"Warning: File extension may not be supported: {video_path}")

        first_frame_rgb = None
        first_blobs = None
        last_frame_rgb = None
        last_blobs = None

        # max-particles frame tracking
        max_frame_rgb = None
        max_blobs = None
        max_red_count = -1
        max_frame_index_original = None
        max_frame_index_used = None

        # iterate frames (this uses your iter_video_frames and detect_red_blobs, overlay_blobs)
        for kept_i, orig_i, frame_rgb, fps in tqdm(
            iter_video_frames(video_path, every_n=PROCESS_EVERY_N_FRAMES, max_frames=MAX_FRAMES_TO_ANALYZE),
            total=total_frames,
            desc="Analyzing frames"):

            blobs = detect_red_blobs(frame_rgb)

            # keep first + last for arrow visualization later
            if first_frame_rgb is None:
                first_frame_rgb = frame_rgb.copy()
                first_blobs = list(blobs)
            last_frame_rgb = frame_rgb.copy()
            last_blobs = list(blobs)

            # update max frame if this frame has more red particles
            if len(blobs) > max_red_count:
                max_red_count = len(blobs)
                max_frame_rgb = frame_rgb.copy()
                max_blobs = list(blobs)
                max_frame_index_original = orig_i
                max_frame_index_used = kept_i

            time_sec = orig_i / fps

            # ---- per-blob feature export ----
            for bi, blob in enumerate(blobs):
                blob_rows.append({
                    "frame_index_original": orig_i,
                    "frame_index_used": kept_i,
                    "time_sec": time_sec,
                    "blob_id_in_frame": bi,
                    **blob
                })

            results.append({
                "frame_index_original": orig_i,
                "frame_index_used": kept_i,
                "time_sec": time_sec,
                "n_red_particles": len(blobs)
            })

            annotated = overlay_blobs(frame_rgb, blobs)
            annotated_frames.append(annotated)  # collect frames for cell7 writer

            # Write annotated frame immediately (still open writer to save on-the-fly)
            if writer is None:
                h, w = annotated.shape[:2]
                out_fps = fps / PROCESS_EVERY_N_FRAMES
                fourcc = cv2.VideoWriter_fourcc(*"mp4v")
                writer = cv2.VideoWriter(annotated_mp4_path, fourcc, out_fps, (w, h))
            fr_bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
            writer.write(fr_bgr)

        # Close writer if opened
        if writer is not None:
            writer.release()
            annotated_video_written = True
            print("Saved annotated video (during analysis):", annotated_mp4_path)

        # Handle empty video / no analyzed frames
        if len(results) == 0:
            raise RuntimeError("No frames were analyzed. Check video path and settings.")

        # Save summary CSV
        df = pd.DataFrame(results)
        df.to_csv(csv_path, index=False)
        print("Saved CSV:", csv_path)

        # Save per-blob features
        df_blobs = pd.DataFrame(blob_rows)
        blobs_csv_path = os.path.join(out_dir, "red_blob_features.csv")
        df_blobs.to_csv(blobs_csv_path, index=False)
        print("Saved per-blob features CSV:", blobs_csv_path)

        # Save run configuration for reproducibility
        run_config = {
            "video_path": video_path,
            "PROCESS_EVERY_N_FRAMES": PROCESS_EVERY_N_FRAMES,
            "MAX_FRAMES_TO_ANALYZE": MAX_FRAMES_TO_ANALYZE,
            "BG_BLUR_SIGMA": BG_BLUR_SIGMA,
            "BACKGROUND_METHOD": BACKGROUND_METHOD,
            "ROLLING_BALL_RADIUS": ROLLING_BALL_RADIUS,
            "RED_DOM_RATIO": RED_DOM_RATIO,
            "RED_MIN_ABS": RED_MIN_ABS,
            "LOG_SIGMA_MIN": LOG_SIGMA_MIN,
            "LOG_SIGMA_MAX": LOG_SIGMA_MAX,
            "LOG_NUM_SIGMAS": LOG_NUM_SIGMAS,
            "LOG_THRESH": LOG_THRESH,
            "ROBUST_K": ROBUST_K,
            "MIN_PARTICLE_AREA": MIN_PARTICLE_AREA,
            "MAX_PARTICLE_AREA": MAX_PARTICLE_AREA,
            "SNR_MIN": SNR_MIN,
        }
        with open(os.path.join(out_dir, "run_config.json"), "w") as f:
            json.dump(run_config, f, indent=2)
        print("Saved run config:", os.path.join(out_dir, "run_config.json"))

        # print("Analysis completed.")

        # return all needed state for following UI/saving code
        return {
            "annotated_video_written": annotated_video_written,
            "annotated_frames": annotated_frames,
            "df": df,
            "df_blobs": df_blobs,
            "out_dir": out_dir,
            "annotated_mp4_path": annotated_mp4_path,
            "csv_path": csv_path,
            "blobs_csv_path": blobs_csv_path,
            "video_path": video_path,
        }

    def single_spot(state):
        """
        Interactive single-spot (blob) intensity-gradient analysis.
        """
        print(" ")
        print("_______________________________")
        print("single-spot Analysis")

        # ---- globals needed ----
        global df_blobs, out_dir

        import os, math
        import numpy as np
        import pandas as pd
        import cv2
        import matplotlib.pyplot as plt
        import ipywidgets as widgets
        from IPython.display import display, clear_output
        from matplotlib.patches import Circle   # <-- FIX 1

        # ---- ensure df_blobs exists ----
        if df_blobs is None or len(df_blobs) == 0:
            blobs_csv_path = os.path.join(out_dir, "red_blob_features.csv")
            if not os.path.exists(blobs_csv_path):
                raise FileNotFoundError(
                    f"Per-blob features file not found: {blobs_csv_path}"
                )
            df_blobs = pd.read_csv(blobs_csv_path)

        # ---- helper: load frame ----
        def load_frame_by_original_index(path, orig_index):
            cap = cv2.VideoCapture(path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(orig_index))
            ok, frame_bgr = cap.read()
            cap.release()
            if not ok:
                raise ValueError(f"Cannot read frame {orig_index}")
            return cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

        # ---- use last analyzed frame ----
        frame_ref = int(df_blobs["frame_index_original"].max())
        frame_blobs = df_blobs[df_blobs["frame_index_original"] == frame_ref].reset_index(drop=True)
        frame_rgb = load_frame_by_original_index(video_path, frame_ref)

        # ---- widgets ----
        blob_slider = widgets.IntSlider(
            value=0, min=0, max=len(frame_blobs) - 1,
            description="Blob index", continuous_update=True
        )

        time_unit_dropdown = widgets.Dropdown(
            options=[("Seconds", "sec"), ("Minutes", "min")],
            value="sec", description="Time unit"
        )

        run_button = widgets.Button(description="Compute gradient", button_style="primary")
        preview_out = widgets.Output()
        result_out = widgets.Output()

        # ---- preview ----
        def update_preview(change=None):
            idx = blob_slider.value
            row_sel = frame_blobs.iloc[idx]

            with preview_out:
                clear_output(wait=True)
                fig, ax = plt.subplots(figsize=(6, 6))
                ax.imshow(frame_rgb)
                ax.axis("off")
                ax.set_title(f"Frame {frame_ref} | blob {idx}")

                for _, r in frame_blobs.iterrows():
                    ax.add_patch(Circle((r["x"], r["y"]), r["radius"],
                                        edgecolor="lime", facecolor="none", lw=1))
                ax.add_patch(Circle((row_sel["x"], row_sel["y"]), row_sel["radius"],
                                    edgecolor="white", facecolor="none", lw=2))
                plt.show()

        # ---- compute gradient ----
        def compute_gradient_for_selected_blob(b):
            idx = blob_slider.value
            row0 = frame_blobs.iloc[idx]

            x0, y0, r0 = float(row0["x"]), float(row0["y"]), float(row0["radius"])
            max_dist = max(2 * r0, 8.0)

            times, intensities = [], []

            for f in sorted(df_blobs["frame_index_original"].unique()):
                df_f = df_blobs[df_blobs["frame_index_original"] == f]
                if df_f.empty:
                    continue
                d2 = (df_f["x"] - x0) ** 2 + (df_f["y"] - y0) ** 2
                i = d2.idxmin()
                if math.sqrt(d2.loc[i]) > max_dist:
                    continue
                times.append(float(df_f.loc[i, "time_sec"]))
                intensities.append(float(df_f.loc[i, "mean_r"]))

            with result_out:
                clear_output(wait=True)
                if len(times) < 2:
                    print("Not enough points for gradient.")
                    return

                t = np.array(times)
                I = np.array(intensities)

                if time_unit_dropdown.value == "min":
                    t = t / 60.0
                    xlabel = "Time (min)"
                    unit = "min"
                else:
                    xlabel = "Time (sec)"
                    unit = "sec"

                dI = np.diff(I)
                dt = np.diff(t)
                grad = dI / dt
                t_mid = (t[:-1] + t[1:]) / 2

                slope, _ = np.polyfit(t, I, 1)
                print(f"Global dI/dt ≈ {slope:.4f} intensity / {unit}")

                fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 6), sharex=True)
                ax1.plot(t, I, marker="o")
                ax1.set_title("Selected spot: intensity vs time")
                ax1.set_ylabel("Mean red intensity")
                ax1.grid(True)

                colors = ["green" if g > 0 else "red" if g < 0 else "gray" for g in grad]
                ax2.bar(t_mid, grad, color=colors, width=np.median(dt) * 0.8)
                ax2.set_title("Gradient of red signal vs time\n(red = dimming, green = brightening, gray = stable)")
                ax2.axhline(0, ls="--")
                ax2.set_xlabel(xlabel)
                ax2.set_ylabel(f"dI/dt (intensity / {unit})")
                ax2.grid(True)

                plt.tight_layout()
                plt.show()

        # ---- FIX 2: UI wiring MUST be here (not inside compute fn) ----
        blob_slider.observe(update_preview, names="value")
        run_button.on_click(compute_gradient_for_selected_blob)

        update_preview()
        controls = widgets.HBox([blob_slider, time_unit_dropdown, run_button])
        display(widgets.VBox([controls, preview_out, result_out]))


    def run_save_and_plot(state):
        """
        Replaces original Cell 7 save/plot UI. Accepts 'state' returned by run_red_particle_analysis().
        """
        # unpack state safely
        annotated_video_written = state.get("annotated_video_written", False)
        annotated_frames = state.get("annotated_frames", [])
        df = state.get("df", pd.DataFrame())
        df_blobs = state.get("df_blobs", pd.DataFrame())

        # safe fallback for out_dir and annotated_mp4_path using globals()
        out_dir = state.get("out_dir")
        if out_dir is None:
            out_dir = globals().get('out_dir', ".")
        annotated_mp4_path = state.get("annotated_mp4_path")
        if annotated_mp4_path is None:
            annotated_mp4_path = globals().get('annotated_mp4_path', os.path.join(out_dir, "annotated.mp4"))

        # try to use poisson if available, else None
        try:
            poisson  # noqa: F401
        except NameError:
            poisson = None

        # ---- If annotated video wasn't written during analysis, write it now from annotated_frames ----
        if (not annotated_video_written) and len(annotated_frames) > 0:
            h, w = annotated_frames[0].shape[:2]
            cap_tmp = cv2.VideoCapture(video_path)
            fps0 = cap_tmp.get(cv2.CAP_PROP_FPS)
            if not fps0 or fps0 < 1:
                print("Warning: Could not determine FPS, using 25.0 as default")
                fps0 = 25.0
            cap_tmp.release()
            out_fps = fps0 / PROCESS_EVERY_N_FRAMES

            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(annotated_mp4_path, fourcc, out_fps, (w, h))

            for fr in annotated_frames:
                fr_bgr = cv2.cvtColor(fr, cv2.COLOR_RGB2BGR)
                writer.write(fr_bgr)
            writer.release()
            print("Saved annotated video:", annotated_mp4_path)

        # ---- Interactive time-axis scaling (widgets + plotting) ----
        import ipywidgets as widgets
        from IPython.display import display, clear_output

        # Dynamically calculated defaults from video timing
        _df_interval_min = float(df["time_sec"].diff().median() / 60.0) if len(df) > 1 else 0.0
        _df_total_min = float(df["time_sec"].iloc[-1] / 60.0) if len(df) > 0 else 0.0

        # Use global defaults if set, otherwise use dynamically calculated values
        default_interval_min = globals().get('DEFAULT_INTERVAL_MIN') if (globals().get('DEFAULT_INTERVAL_MIN') is not None) else _df_interval_min
        default_total_min = globals().get('DEFAULT_TOTAL_OBS_MIN') if (globals().get('DEFAULT_TOTAL_OBS_MIN') is not None) else _df_total_min

        w_total_min = widgets.FloatText(
            value=round(default_total_min, 3),
            description="Total obs (min):",
            layout=widgets.Layout(width="240px")
        )
        w_interval_min = widgets.FloatText(
            value=round(default_interval_min, 4),
            description="Interval (min):",
            layout=widgets.Layout(width="220px")
        )

        w_mode = widgets.ToggleButtons(
            options=[
                ("Fit to total time", "fit_total"),
                ("Use interval", "use_interval"),
                ("Plot at obs intervals", "bin_interval"),
            ],
            value="fit_total",
            description="Mode:",
            layout=widgets.Layout(width="520px")
        )

        w_agg = widgets.Dropdown(
            options=[("Mean", "mean"), ("Max", "max"), ("Median", "median"), ("Sum", "sum")],
            value="mean",
            description="Aggregate:",
            layout=widgets.Layout(width="220px")
        )

        btn_time_apply = widgets.Button(description="Apply time scale", button_style="info", icon="clock")
        out_timeplot = widgets.Output()

        def _base_time_axis_minutes():
            """Continuous time axis for each analyzed frame."""
            n = len(df)
            total_min = float(w_total_min.value or 0.0)
            interval_min = float(w_interval_min.value or 0.0)
            mode = w_mode.value

            if n == 0:
                return np.array([]), 0.0, 0.0, "no data"

            if (mode in ["fit_total", "bin_interval"]) and total_min > 0:
                t_min = np.linspace(0, total_min, n)
                interval_eff = total_min / (n - 1) if n > 1 else 0.0
                return t_min, total_min, interval_eff, "fit_total_base"

            if interval_min > 0:
                t_min = np.arange(n) * interval_min
                total_eff = float(t_min[-1]) if n > 0 else 0.0
                return t_min, total_eff, interval_min, "use_interval_base"

            # fallback to fps-derived time
            t_min = df["time_sec"].values / 60.0
            interval_eff = float(np.median(np.diff(t_min))) if n > 1 else 0.0
            total_eff = float(t_min[-1]) if n > 0 else 0.0
            return t_min, total_eff, interval_eff, "video_fps_base"

        def _aggregate_to_intervals(t_min, interval_min, total_min):
            """Bin per-frame data into observation intervals."""
            if interval_min <= 0:
                return t_min, df["n_red_particles"].values, "interval not set"

            # decide end time
            end_t = total_min if total_min > 0 else float(t_min[-1])
            if end_t <= 0:
                end_t = float(t_min[-1])

            # build bin edges and labels
            edges = np.arange(0, end_t + interval_min*0.999, interval_min)
            if len(edges) < 2:
                edges = np.array([0, end_t])

            bin_ids = np.digitize(t_min, edges, right=False) - 1
            nbins = len(edges) - 1
            agg_vals = []
            agg_t = []
            vals = df["n_red_particles"].values

            for b in range(nbins):
                mask = bin_ids == b
                if not np.any(mask):
                    continue
                v = vals[mask]
                if w_agg.value == "mean":
                    a = float(np.mean(v))
                elif w_agg.value == "max":
                    a = float(np.max(v))
                elif w_agg.value == "median":
                    a = float(np.median(v))
                else:
                    a = float(np.sum(v))
                agg_vals.append(a)
                agg_t.append(edges[b])  # left edge as observation time

            return np.array(agg_t), np.array(agg_vals), "binned"

        def _draw_timeplot(*args):
            with out_timeplot:
                clear_output(wait=True)
                t_min, total_eff, interval_eff, base_mode = _base_time_axis_minutes()
                mode = w_mode.value
                n = len(df)

                y = df["n_red_particles"].values
                t_plot = t_min
                y_plot = y
                note = ""

                if mode == "bin_interval":
                    interval_min_in = float(w_interval_min.value or 0.0)
                    total_min_in = float(w_total_min.value or 0.0)
                    t_plot, y_plot, note = _aggregate_to_intervals(t_min, interval_min_in, total_min_in)

                plt.figure(figsize=(8,4))
                sns.lineplot(x=t_plot, y=y_plot, marker="o", linewidth=1)
                # ---- uncertainty band (approx. 95% CI) ----
                if poisson is not None:
                    ci_low = poisson.ppf(0.025, y_plot)
                    ci_high = poisson.ppf(0.975, y_plot)
                else:
                    ci_low = np.maximum(0, y_plot - 1.96*np.sqrt(np.maximum(y_plot, 0)))
                    ci_high = y_plot + 1.96*np.sqrt(np.maximum(y_plot, 0))
                plt.fill_between(t_plot, ci_low, ci_high, alpha=0.2)

                plt.xlabel("Time (min)")
                plt.ylabel("Red particle count / frame")
                title = "Red particles vs time (scaled to experiment)"
                if mode == "bin_interval":
                    title += f" - {w_agg.label} per interval"
                plt.title(title)
                plt.grid(True)

                # Add custom x-axis ticks for 30, 60, 90... min intervals
                if len(t_plot) > 0:
                    min_t = np.min(t_plot)
                    max_t = np.max(t_plot)
                    # Choose a reasonable tick interval, e.g., 30 minutes, or 10 if shorter duration
                    tick_interval = 30
                    if max_t < 60:
                        tick_interval = 10
                    elif max_t < 180:
                        tick_interval = 30
                    else:
                        tick_interval = 60

                    # Generate ticks from 0 up to max_t, at the chosen interval
                    xticks = np.arange(0, max_t + tick_interval, tick_interval)
                    plt.xticks(xticks)

                plt.show()

                if mode == "use_interval" and float(w_total_min.value or 0) > 0 and float(w_interval_min.value or 0) > 0:
                    total_in = float(w_total_min.value)
                    expected_total = (n - 1) * float(w_interval_min.value)
                    if abs(expected_total - total_in) > 1e-6:
                        print(f"Note: (N-1)*interval = {expected_total:.2f} min differs from Total obs = {total_in:.2f} min.")
                        print("Switch to 'Fit to total time' or 'Plot at obs intervals' if needed.")

                if mode == "bin_interval" and note == "interval not set":
                    print("Set Interval (min) to enable observation-interval plotting.")

                print(f"Mode: {mode} | base={base_mode} | N={n} | interval/frame~{interval_eff:.4g} min | total~{total_eff:.4g} min")

        btn_time_apply.on_click(_draw_timeplot)

        display(widgets.VBox([
            widgets.HBox([w_total_min, w_interval_min, w_agg]),
            w_mode,
            btn_time_apply
        ]))
        display(out_timeplot)

        # initial draw
        _draw_timeplot()

        # After initial draw, capture the currently displayed (or calculated) defaults
        # Save back to globals so subsequent runs will use these defaults
        try:
            globals()['DEFAULT_TOTAL_OBS_MIN'] = float(w_total_min.value)
            globals()['DEFAULT_INTERVAL_MIN'] = float(w_interval_min.value)
        except Exception:
            pass

        # ---- Scroll through analyzed frames with white arrows pointing to detected red particles ----

        def show_frame_with_arrows(frame_rgb, blobs, title):
            if frame_rgb is None:
                print(f"{title}: frame not available")
                return
            plt.figure(figsize=(7, 7))
            plt.imshow(frame_rgb)
            ax = plt.gca()
            if blobs is not None:
                for blob in blobs:
                    y = float(blob["y"])
                    x = float(blob["x"])
                    # arrow from offset to blob center
                    ax.annotate(
                        "",
                        xy=(x, y),
                        xytext=(x + 15, y - 15),
                        arrowprops=dict(arrowstyle="->", color="white", lw=1.8),
                    )
            ax.set_title(title)
            plt.axis("off")
            plt.show()

        def load_frame_by_original_index_for_viewer(path, orig_index):
            """Lightweight helper for the frame-scrolling viewer."""
            cap = cv2.VideoCapture(path)
            if not cap.isOpened():
                raise IOError(f"Cannot open video: {path}")
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(orig_index))
            ok, frame_bgr = cap.read()
            cap.release()
            if not ok:
                raise ValueError(f"Could not read frame {orig_index} from {path}")
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            return frame_rgb

        # Collect the original frame indices that were analyzed
        try:
            frame_indices = sorted(df["frame_index_original"].unique().tolist())
        except Exception:
            frame_indices = []

        if len(frame_indices) > 0:
            frame_slider = widgets.IntSlider(
                value=int(frame_indices[0]),
                min=int(frame_indices[0]),
                max=int(frame_indices[-1]),
                step=PROCESS_EVERY_N_FRAMES,
                description="Frame index:",
                continuous_update=False,
                readout=True,
                layout=widgets.Layout(width="600px"),
            )

            scroll_out = widgets.Output()

            def _update_scroll_view(change=None):
                idx = frame_slider.value
                # Load raw frame from the original video
                frame_rgb = load_frame_by_original_index_for_viewer(video_path, idx)

                # Get blobs for this frame from the per-blob table
                rows = df_blobs[df_blobs["frame_index_original"] == idx]
                blobs_for_frame = [
                    {"x": r["x"], "y": r["y"], "radius": r["radius"]}
                    for _, r in rows.iterrows()
                ]

                with scroll_out:
                    clear_output(wait=True)
                    if frame_rgb is None:
                        print(f"Frame {idx}: not available")
                        return
                    if len(blobs_for_frame) == 0:
                        title = f"Frame {idx} (no red particles detected)"
                    else:
                        title = f"Frame {idx} (n_red={len(blobs_for_frame)})"
                    show_frame_with_arrows(frame_rgb, blobs_for_frame, title)

            frame_slider.observe(_update_scroll_view, names="value")
            _update_scroll_view()

            display(widgets.VBox([
                widgets.HTML("<b>Scroll through frames with white arrows on detected red particles</b>"),
                frame_slider,
                scroll_out,
            ]))
        else:
            print("No analyzed frames available for the scrolling viewer.")

        globals()['df_blobs'] = df_blobs
        globals()['out_dir'] = out_dir

        # print("Running single spot analysis")
        single_spot(state)



    btn_preview.on_click(_do_preview)
    btn_run.on_click(_run_analysis)

    display(widgets.VBox([
        widgets.HTML("<b>Tune sensitivity / size:</b>"),
        widgets.HBox([w_bg_sigma, w_red_dom, w_red_abs]),
        widgets.HBox([w_sig_min, w_sig_max, w_sig_num]),
        widgets.HBox([w_log_thr, w_rob_k]),
        widgets.HBox([w_area_min, w_area_max]),
        widgets.HBox([btn_preview, btn_run]),
        out,
        download_ui_out
    ]))

    # auto-run first preview
    _do_preview()

# ------------------------------------------------------------
# CASE 1: Google Colab
# ------------------------------------------------------------
if _is_colab():
    from google.colab import files
    print(" ")
    print("_______________________________")
    print("Colab detected. Click 'Choose Files' to upload an MP4.")
    uploaded = files.upload()

    if len(uploaded) == 0:
        raise RuntimeError("No file uploaded.")

    name = list(uploaded.keys())[0]
    video_path = f"/content/{name}"

    print("Uploaded:", video_path)

    # Run setting automatically
    try:
        run_after_upload(video_path)
    except Exception as e:
        print("Error in Setting:", e)




# ------------------------------------------------------------
# CASE 2: Jupyter Notebook / Local / Kaggle
# ------------------------------------------------------------
else:
    import ipywidgets as widgets
    from IPython.display import display
    import os

    uploader = widgets.FileUpload(accept=".mp4", multiple=False)
    display(uploader)

    def _save_upload_and_run(change=None):
        global video_path

        if len(uploader.value) == 0:
            print("Please upload an MP4, then re-run this cell.")
            return

        item = next(iter(uploader.value.values()))
        name = item["metadata"]["name"]
        data = item["content"]

        save_dir = "/mnt/data" if os.path.exists("/mnt/data") else os.getcwd()
        save_path = os.path.join(save_dir, name)

        with open(save_path, "wb") as f:
            f.write(data)

        video_path = save_path
        print("Saved:", video_path)

        # Run setting automatically
        try:
            run_after_upload(video_path)
        except Exception as e:
            print("Error in Setting:", e)

    uploader.observe(_save_upload_and_run, names="value")

 
_______________________________
Colab detected. Click 'Choose Files' to upload an MP4.


Saving WT diff zymosan 03-2.mp4 to WT diff zymosan 03-2.mp4
Uploaded: /content/WT diff zymosan 03-2.mp4
 
_______________________________
Setting
Video OK: 20 frames, 5.000 fps


# **ANALYSIS**

In [4]:
run_analysis()

 
_______________________________
Running Analysis ...
